In [150]:
# using Pkg
# Pkg.add("Distributions")

# Probit-SUE: an illustration of fixed point formulation with MSA as the solution algorithm.

In [151]:
using DelimitedFiles:readdlm
using Distributions:Normal, MvNormal
import Random

## Problem setting.

<img src="data/Caliper_net.png" width="800">

In [152]:
netFile = "data/network.csv"
csvdata = readdlm(netFile, ',', header=true)
data = csvdata[1]
header = csvdata[2]

1×10 Matrix{AbstractString}:
 "tail"  "head"  "cap"  "length"  "fft"  …  "speedLimit"  "toll"  "link_type"

In [153]:
display(header)
display(data)

1×10 Matrix{AbstractString}:
 "tail"  "head"  "cap"  "length"  "fft"  …  "speedLimit"  "toll"  "link_type"

7×10 Matrix{Float64}:
 1.0  2.0  50.0  20.0  20.0  0.15  4.0  60.0  0.0  1.0
 1.0  4.0  50.0  20.0  20.0  0.15  4.0  60.0  0.0  1.0
 2.0  3.0  50.0  20.0  20.0  0.15  4.0  60.0  0.0  1.0
 2.0  5.0  50.0  20.0  20.0  0.15  4.0  60.0  0.0  1.0
 3.0  6.0  50.0  20.0  20.0  0.15  4.0  60.0  0.0  1.0
 4.0  5.0  50.0  20.0  10.0  0.15  4.0  60.0  4.0  1.0
 5.0  6.0  50.0  20.0  10.0  0.15  4.0  60.0  2.0  1.0

In [154]:
links = []
for l in zip(round.(Int, data[:,1]), round.(Int, data[:,2]))
    push!(links, l)
end
cap = data[:, 3]
fft = data[:, 5]
alpha = data[:, 6]
beta = data[:, 7]
dem = 100.0

100.0

In [155]:
# BPR function
function BPR(cap, fft, alpha, beta,  flow)
    return fft * (1 + alpha * (flow/cap) ^ beta)
end

BPR (generic function with 1 method)

### The perception errors follow a multivariate normal distribution with $\mathbf{\mu=0}$ and zero covariance.

In [156]:
# Multivariate normal distribution
μ = [0.0; 0.0; 0.0]
Σ = Float64[
    10 0 0;
    0 10 0;
    0 0 10;
    ]

Random.seed!(123)
MVN = MvNormal(μ, Σ)
samples = rand(MVN, 1000)

3×1000 Matrix{Float64}:
 -2.04198  -0.688318   0.252846   1.30337  …  -1.10809    6.26602   2.62077
 -4.62721   1.55662    4.89876    1.87585     -0.417665   9.16882   0.736885
 -5.13429   3.10213   -4.24254   -2.42992     -3.93141   -4.91512  -0.979004

In [157]:
# Probit probability using Monte-Carlo simulation
function Probit(cost)
    count = [0; 0; 0]
    for error in eachcol(samples)
        util = -(cost + error)
        count[argmax(util)] += 1
    end
    return count / sum(count)
end

# test the Probit probability
cost = [21.0, 21.1, 21.9]
Probit(cost)

3-element Vector{Float64}:
 0.392
 0.349
 0.259

In [158]:
# link-path incidence matrix (a 7 x 3 matrix)
display(links)
Δ = Matrix([
    1 1 0;
    0 0 1;
    1 0 0;
    0 1 0;
    1 0 0;
    0 0 1;
    0 1 1
    ])

7-element Vector{Any}:
 (1, 2)
 (1, 4)
 (2, 3)
 (2, 5)
 (3, 6)
 (4, 5)
 (5, 6)

7×3 Matrix{Int64}:
 1  1  0
 0  0  1
 1  0  0
 0  1  0
 1  0  0
 0  0  1
 0  1  1

## Formulation
It is a fixed point problem with respect to path flow. 
$$
\mathbf{f} = F(\mathbf{f}) \Rightarrow \mathbf{f = q * Pr(-\Delta^{T}t(\Delta f))}
$$

Alternatively, we can also write it as a fixed point problem with respect to link flow.
$$
\mathbf{x} = G(\mathbf{x}) \Rightarrow \mathbf{x = \Delta q * Pr(-\Delta^{T}t(x))}
$$

We use MSA method to solve this fixed point problem with respect to path flow. 

In [159]:
# Let's do it for many loops.
f = [100; 0; 0]
f_aux = []
for iter in 1:1000
    f_aux = dem * Probit(Δ' * BPR.(cap, fft, alpha, beta, Δ * f))
    f = (f_aux  + iter * f) / (1 + iter)
end
println("f_aux = $f_aux")
println("f     = $f")
println("x     = $(Δ*f)")

f_aux = [20.4, 20.4, 59.199999999999996]
f     = [20.338361638361405, 20.340459540459346, 59.321178821179124]
x     = [40.678821178820755, 59.321178821179124, 20.338361638361405, 20.340459540459346, 20.338361638361405, 59.321178821179124, 79.66163836163847]


### Larger perception errors will equalize the paths.

In [160]:
# Multivariate normal distribution
μ = [0.0; 0.0; 0.0]
Σ = Float64[
    10000 0 0;
    0 10000 0;
    0 0 10000;
    ]

Random.seed!(123)
MVN = MvNormal(μ, Σ)
samples = rand(MVN, 1000)

3×1000 Matrix{Float64}:
  -64.5731  -21.7665     7.99568   41.2162  …   -35.041    198.149   82.8762
 -146.325    49.2246   154.912     59.3197      -13.2077   289.944   23.3024
 -162.36     98.098   -134.161    -76.8409     -124.322   -155.43   -30.9588

In [161]:
# Let's do it for many loops.
f = [100; 0; 0]
f_aux = []
for iter in 1:1000
    f_aux = dem * Probit(Δ' * BPR.(cap, fft, alpha, beta, Δ * f))
    f = (f_aux  + iter * f) / (1 + iter)
end
println("f_aux = $f_aux")
println("f     = $f")
println("x     = $(Δ*f)")

f_aux = [32.0, 29.7, 38.3]
f     = [32.0235764235764, 29.677422577423066, 38.29900099900104]
x     = [61.70099900099947, 38.29900099900104, 32.0235764235764, 29.677422577423066, 32.0235764235764, 38.29900099900104, 67.97642357642411]


### The perception errors follow a multivariate normal distribution with $\mathbf{\mu=0}$ and non-zero covariance.

In [162]:
# Multivariate normal distribution
μ = [0.0; 0.0; 0.0]
Σ = Float64[
    10 5 0;
    5 10 5;
    0 5 10;
    ]

Random.seed!(123)
MVN = MvNormal(μ, Σ)
samples = rand(MVN, 1000)

3×1000 Matrix{Float64}:
 -2.04198  -0.688318   0.252846   1.30337   …  -1.10809    6.26602   2.62077
 -5.02827   1.00391    4.36888    2.27622      -0.915756  11.0734    1.94855
 -6.86365   3.43159   -0.635719  -0.900998     -3.45112    1.28044  -0.373912

In [163]:
# Let's do it for many loops.
f = [100; 0; 0]
f_aux = []
for iter in 1:1000
    f_aux = dem * Probit(Δ' * BPR.(cap, fft, alpha, beta, Δ * f))
    f = (f_aux  + iter * f) / (1 + iter)
end
println("f_aux = $f_aux")
println("f     = $f")
println("x     = $(Δ*f)")

f_aux = [21.4, 18.3, 60.3]
f     = [21.35254745254736, 18.30009990009949, 60.347352647353674]
x     = [39.65264735264685, 60.347352647353674, 21.35254745254736, 18.30009990009949, 21.35254745254736, 60.347352647353674, 78.64745254745316]


If we consider the covariance (i.e., overlapping), less people choose path 2.